In [1]:
from chembl_webresource_client.new_client import new_client
from chembl_webresource_client.query_set import QuerySet
import pandas as pd
from os import mkdir

try:
  mkdir("results")
except:
  pass

In [22]:
# type: ignore

def CleanedActivitiesDF(data: pd.DataFrame) -> pd.DataFrame:
    try:
        data = data.drop(['activity_id', 'activity_properties',
                          'document_journal', 'document_year',
                          'molecule_pref_name', 'pchembl_value',
                          'potential_duplicate', 'qudt_units',
                          'record_id', 'src_id', 'standard_flag',
                          'standard_text_value', 'standard_upper_value',
                          'target_chembl_id', 'target_pref_name',
                          'target_tax_id', 'text_value', 'toid',
                          'type', 'units', 'uo_units', 'upper_value',
                          'value'], axis=1)

    except Exception as exception:
        print(f"{exception}".ljust(77))

    try:
        data = data[data['relation'] == '=']
        data = data[data['standard_units'] == 'nM']
        data = data[data['target_organism'] == "Homo sapiens"]
        data = data[data['standard_type'].isin(['IC50','Ki'])]

        data = data.drop(['target_organism', 'standard_type'], axis=1)
    except Exception as exception:
        print(f"{exception}".ljust(77))


    # TODO: произвести фильтрацию:
    # для повторов по molecule_chembl_id - считать медиану nM
    # ? скачать по molecule_chembl_id также информацию о targets

    try:
        data = data.reindex(columns=["molecule_chembl_id", "parent_molecule_chembl_id",
                                 "canonical_smiles", "document_chembl_id", "standard_relation",
                                 "relation", "standard_value", "standard_units", 'assay_chembl_id',
                                 'assay_description', 'assay_type', 'assay_variant_accession',
                                 'assay_variant_mutation', 'action_type', 'activity_comment',
                                 'data_validity_comment', 'data_validity_description',
                                 'bao_endpoint', 'bao_format', 'bao_label', 'ligand_efficiency'])
        
    except Exception as exception:
        print(f"{exception}".ljust(77))
    
    return data

In [23]:
# type: ignore

targs_ids = new_client.target.filter().only('target_chembl_id')[0]                               
activities = new_client.activity.filter(target_chembl_id=targs_ids['target_chembl_id']).filter(standard_type="IC50")

df = CleanedActivitiesDF(pd.DataFrame(activities))
df

,molecule_chembl_id,parent_molecule_chembl_id,canonical_smiles,document_chembl_id,standard_relation,relation,standard_value,standard_units,assay_chembl_id,assay_description,...,assay_variant_accession,assay_variant_mutation,action_type,activity_comment,data_validity_comment,data_validity_description,bao_endpoint,bao_format,bao_label,ligand_efficiency
0,CHEMBL2114210,CHEMBL2114210,OC[C@H]1N[C@H](CO)[C@@H](O)C(O)[C@@H]1O,CHEMBL1131015,=,=,110000.0,nM,CHEMBL649260,Inhibitory activity measured against alpha-glu...,...,None,None,None,None,Outside typical range,Values for this activity type are unusually la...,BAO_0000190,BAO_0000357,single protein format,None
1,CHEMBL87169,CHEMBL87169,OCC1NC(CO)[C@@H](O)C(O)C1O,CHEMBL1131015,=,=,700.0,nM,CHEMBL649260,Inhibitory activity measured against alpha-glu...,...,None,None,None,None,None,None,BAO_0000190,BAO_0000357,single protein format,"{'bei': '31.86', 'le': '0.65', 'lle': '9.76', ..."
2,CHEMBL2115215,CHEMBL2115215,OC[C@@H]1N[C@H](CO)[C@@H](O)[C@H](O)[C@@H]1O,CHEMBL1131015,=,=,3200.0,nM,CHEMBL649260,Inhibitory activity measured against alpha-glu...,...,None,None,None,None,None,None,BAO_0000190,BAO_0000357,single protein format,"{'bei': '28.44', 'le': '0.58', 'lle': '9.10', ..."
3,CHEMBL501355,CHEMBL501355,OC[C@H]1N[C@H](CO)[C@@H](O)[C@H](O)[C@H]1O,CHEMBL1131015,=,=,40.0,nM,CHEMBL649260,Inhibitory activity measured against alpha-glu...,...,None,None,None,None,None,None,BAO_0000190,BAO_0000357,single protein format,"{'bei': '38.29', 'le': '0.78', 'lle': '11.01',..."
4,CHEMBL2093897,CHEMBL2093897,OC[C@@H]1N[C@H](CO)[C@@H](O)[C@H](O)[C@H]1O,CHEMBL1131015,=,=,8400.0,nM,CHEMBL816395,Inhibitory activity measured against alpha-glu...,...,None,None,None,None,None,None,BAO_0000190,BAO_0000357,single protein format,"{'bei': '26.27', 'le': '0.53', 'lle': '8.69', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,CHEMBL3264725,CHEMBL3264725,O=c1oc2ccc(Cl)cc2c(O)c1C(c1cccc(O)c1)c1c(O)c2c...,CHEMBL5260826,=,=,84060.0,nM,CHEMBL5263434,Inhibition of alpha-glucosidase (unknown origin),...,None,None,"{'action_type': 'INHIBITOR', 'description': 'N...",None,None,None,BAO_0000190,BAO_0000357,single protein format,"{'bei': '8.20', 'le': '0.16', 'lle': '-1.42', ..."
356,CHEMBL3264724,CHEMBL3264724,COc1cc(C(c2c(O)c3cc(Cl)ccc3oc2=O)c2c(O)c3cc(Cl...,CHEMBL5260826,=,=,83640.0,nM,CHEMBL5263434,Inhibition of alpha-glucosidase (unknown origin),...,None,None,"{'action_type': 'INHIBITOR', 'description': 'N...",None,None,None,BAO_0000190,BAO_0000357,single protein format,"{'bei': '7.73', 'le': '0.15', 'lle': '-1.43', ..."
357,CHEMBL3264727,CHEMBL3264727,CCOc1cc(C(c2c(O)c3cc(Cl)ccc3oc2=O)c2c(O)c3cc(C...,CHEMBL5260826,=,=,57140.0,nM,CHEMBL5263434,Inhibition of alpha-glucosidase (unknown origin),...,None,None,"{'action_type': 'INHIBITOR', 'description': 'N...",None,None,None,BAO_0000190,BAO_0000357,single protein format,"{'bei': '7.84', 'le': '0.16', 'lle': '-1.66', ..."
358,CHEMBL3264728,CHEMBL3264728,COc1cc(C(c2c(O)c3cc(Cl)ccc3oc2=O)c2c(O)c3cc(Cl...,CHEMBL5260826,=,=,128140.0,nM,CHEMBL5263434,Inhibition of alpha-glucosidase (unknown origin),...,None,None,"{'action_type': 'INHIBITOR', 'description': 'N...",None,Outside typical range,Values for this activity type are unusually la...,BAO_0000190,BAO_0000357,single protein format,None


In [14]:
df.columns

Index(['action_type', 'activity_comment', 'assay_chembl_id',
       'assay_description', 'assay_type', 'assay_variant_accession',
       'assay_variant_mutation', 'bao_endpoint', 'bao_format', 'bao_label',
       'canonical_smiles', 'data_validity_comment',
       'data_validity_description', 'document_chembl_id', 'ligand_efficiency',
       'molecule_chembl_id', 'parent_molecule_chembl_id', 'relation',
       'standard_relation', 'standard_type', 'standard_units',
       'standard_value', 'target_organism'],
      dtype='object')